In [72]:
import os
import numpy as np
from scipy import misc
import argparse
import json
import cv2
import uuid
from tensorpack import *
from train import Model
from reader import Data
from cfgs.config import cfg
import shutil
from matplotlib import pyplot as plt
%matplotlib inline

In [73]:
root_data = "plate_recog_dataset/"
# img_path = "plate_keypoint"
model_path = 'train_log/train0801-090505/model-24816'
# new_plate_path = 'new_plate'
new_plate_h = 50
new_plate_w = 160
pad = 10
refresh =True

In [74]:
# initialize the model
depth = 18
model = Model(depth)
sess_init = SaverRestore(model_path)
predict_config = PredictConfig(session_init=sess_init,
                               model=model,
                               input_names=["input"],
                               output_names=["logits"])

In [75]:
predict_func = OfflinePredictor(predict_config)

[0801 09:38:20 @regularize.py:18] Apply regularizer for conv0/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group0/block0/conv1/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group0/block0/conv2/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group0/block1/conv1/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group0/block1/conv2/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group1/block0/conv1/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group1/block0/conv2/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group1/block0/convshortcut/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group1/block1/conv1/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group1/block1/conv2/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group2/block0/conv1/W:0
[0801 09:38:20 @regularize.py:18] Apply regularizer for group2/block0/conv2/W:0
[0801 09:38:20 @regularize.py:18] Apply regular

In [76]:
def cal_coornidate_function(predictions,ori_img_h,ori_img_w):
#     print(predictions)
    coornidate = []
    temw = (predictions[0]+0.5)*ori_img_w
    temh = (predictions[1]+0.5)*ori_img_h
    coornidate.append([temw,temh])
   
    temw = (predictions[2]+0.5)*ori_img_w
    temh = (predictions[3]+0.5)*ori_img_h
    coornidate.append([temw,temh])
   
    temw = (predictions[4]+0.5)*ori_img_w
    temh = (predictions[5]+0.5)*ori_img_h
    coornidate.append([temw,temh])

    temw = (predictions[6]+0.5)*ori_img_w
    temh = (predictions[7]+0.5)*ori_img_h
    coornidate.append([temw,temh])
#     print(coornidate)
    return coornidate

In [77]:
def Bontpointpredict(img_root,lines,car_warp_path):    
    for idx,line in enumerate(lines):
        if idx % 100 == 0 and idx > 0:
            print(idx)
        img_path = os.path.join(img_root,line)
        print(img_path)
        img = cv2.imread(img_path)
        img_ori = img
    #    plt.imshow(img)
        ori_img_h,ori_img_w = img.shape[0:2]
    #   
#     #     print(img.shape)
#         import pdb
#         pdb.set_trace()
        img = cv2.resize(img, (cfg.img_h, cfg.img_w))
        input_img = np.expand_dims(img, axis=0)
        predictions = predict_func([input_img])[0][0]
        coor = cal_coornidate_function(predictions,ori_img_h,ori_img_w)

   
        pts1 = np.float32(coor)
        pts2 = np.float32([[pad,pad],[pad+new_plate_w,pad],[pad+new_plate_w,pad+new_plate_h],[pad,pad+new_plate_h]])
   
        M = cv2.getPerspectiveTransform(pts1,pts2)
        dst = cv2.warpPerspective(img_ori,M,(new_plate_w+2*pad,new_plate_h+2*pad))
#         plt.imshow(dst)
        img_name = line.split("/")[-1]
#         print(img_name)
        warp_path_tem = os.path.join(car_warp_path,img_name)
#         print(warp_path_tem)
        if os.path.isfile(warp_path_tem) == True:
            os.remove(warp_path_tem)
        cv2.imwrite(warp_path_tem,dst)
#         break

In [78]:
lines = os.listdir(root_data)
cars = [e for e in lines if e.endswith("_label")]
for car in cars:
    car_warp= car.split("_")[0] + "_warp"
    car_warp_path = os.path.join(root_data,car_warp)
    
    if refresh == False and os.path.isdir(car_warp_path):
        continue
    if os.path.isdir(car_warp_path) == True:
        shutil.rmtree(car_warp_path)
    os.mkdir(car_warp_path)
    img_warp_paths = os.path.join(root_data,car)
    img_warps = os.listdir(img_warp_paths)
    Bontpointpredict(img_warp_paths,img_warps,car_warp_path)

#     break
print("process has finishined!!!")

plate_recog_dataset/20170715_label/新nbc988.jpg
plate_recog_dataset/20170715_label/新n28533.jpg
plate_recog_dataset/20170715_label/新ng3590.jpg
plate_recog_dataset/20170715_label/鲁rl4158.jpg
plate_recog_dataset/20170715_label/新mp7765.jpg
plate_recog_dataset/20170715_label/新ne6257.jpg
plate_recog_dataset/20170715_label/新n97758.jpg
plate_recog_dataset/20170715_label/粤a4mu49.jpg
plate_recog_dataset/20170715_label/新n97335.jpg
plate_recog_dataset/20170715_label/新n7c896.jpg
plate_recog_dataset/20170715_label/新nw5136.jpg
plate_recog_dataset/20170715_label/新nz2589.jpg
plate_recog_dataset/20170715_label/新n75388.jpg
plate_recog_dataset/20170715_label/新nz6033.jpg
plate_recog_dataset/20170715_label/新nx5532.jpg
plate_recog_dataset/20170715_label/新na7371.jpg
plate_recog_dataset/20170715_label/新ns3222.jpg
plate_recog_dataset/20170715_label/新nr7123.jpg
plate_recog_dataset/20170715_label/新nh8196.jpg
plate_recog_dataset/20170715_label/新n8b303.jpg
plate_recog_dataset/20170715_label/新nf5367.jpg
plate_recog_d

In [ ]:
# print(len(predictions))


In [ ]:

# plt.savefig('t.jpg')
# plt.show()

In [ ]:
# from reader import Data
# ds = Data("train")
# ds.reset_state()
# producer = ds.get_data()
# dp = next(producer)

In [ ]:
# input_img = np.expand_dims(dp[0], axis=0)

In [88]:
# predictions = predict_func([input_img])[0][0]

In [89]:
# predictions

In [90]:
# plt.imshow(img)

In [91]:
# plt.imshow(dp[0])

In [92]:
# dp[1]